# Connecteren met Google Sheets
We gebruiken een Service Account van Google Cloud die we toegang geven tot de Google Sheet waarin onze gegevens zitten. Dankzij de opgeslagen credentials (key.json) kunnen we rechtstreeks aan de gegevens in de Sheet zonder dat we moeten inloggen.

De sheet moet gedeeld worden met jupyter-google-sheets-service@second-replica-341913.iam.gserviceaccount.com.

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# The scope is always look like this so we did not need to change anything
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
# Name of our Service Account Key
google_key_file = 'assets/key.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)


# Worksheet omzetten naar Pandas Dataframe

In [2]:
import pandas as pd

workbook = gc.open('Enquête Data Science 2022')
sheet = workbook.get_worksheet(0)
values = sheet.get_all_values()
enquete_df = pd.DataFrame(values[1:], columns=values[0])
enquete_df

,Timestamp,1. Wat was je score op de GapMinder test? Je hoeft je niet te schamen als het slecht was :-). De meeste mensen score vrij slecht. Noteer de score hieronder als een breuk.,2. Met welke hand schrijf je?,3. Wat is je schoenmaat?,4. Wat is je lichaamslengte (in cm)?,5. Wat is je bloedgroep: AB0?,6. Wat is de resusfactor van je bloedgroep?,7. Hoeveel broers / zussen heb je?,8. Vink alle beweringen aan die volgens aannemelijk zijn.,9. Welke rijbewijzen heb je al volledig behaald?,...,23c. Weerloze wezens moeten beschermd worden (0 helemaal oneens – 10 helemaal eens),23d. Je kunt best altijd het voorbeeld van een professional volgen (0 helemaal oneens – 10 helemaal eens),23e. Wetenschappelijk onderzoek op ongeboren embryo’s is noodzakelijk (0 helemaal oneens – 10 helemaal eens),23f. Vrijheid is belangrijker dan gelijkheid (0 helemaal oneens – 10 helemaal eens),"24. Welk zoogdier zou je als klasmascotte verkiezen? Je hebt de keuze uit: Panter, Poema, Vos, Wolf, Bruine beer, Bever, Das, Bizon, Wasbeer, Otter. Sorteer ze van 1 tot 10.",25. Wat is volgens jou het zwaarste vak uit het eerste jaar?,26. Welke vak uit het eerste jaar TI vind je het meest boeiend?,27. In het hoger onderwijs wordt 1 studiepunt gelijkgesteld aan 25 uren studiebelasting. Wat is jou inschatting?\t,28. Dit is de makkelijkste vraag: je hoeft helemaal niets in te vullen.,29. Ik geef toestemming om deze gegevens -geanonimiseerd- te gebruiken binnen de lessen Data Science van de opleiding Toegepaste Informatica op KdG.
0,10/02/2022 11:33:58,7/18,Rechts,43,184,O,positief,5,De tsunami van 2004 in de Indische Oceaan was ...,Categorie B: Wagens.,...,8,6,5,4,"1=Bever,2=Wasbeer,3=Otter,4=Wolf,5=Poema,6=Pan...",Management 1,Data science 1,16-20 uren,,Ja
1,10/02/2022 11:37:44,3/18,Rechts,42,163,Weet ik niet,Weet ik niet,1,Prinses Diana kwam niet per ongeluk om het leven,Categorie AM: Bromfietsen met een maximumsnelh...,...,10,6,7,3,"1=Bever,2=Wolf,3=Bizon,4=Otter,5=Panter,6=Poem...",Management 1,Computersystemen 1,<= 10 uren,,Ja
2,10/02/2022 11:39:39,12/18,Rechts,45,183,AB,positief,2,Prinses Diana kwam niet per ongeluk om het lev...,Categorie AM: Bromfietsen met een maximumsnelh...,...,7,7,10,6,"1=Bruine beer,2=Poema,3=Panter,4=Wolf,5=Vos,6=...",Management 1,Programmeren 1 - Java,16-20 uren,Oke dank u,Ja
3,10/02/2022 11:40:33,5/18,Rechts,44,175,A,positief,2,De wereld wordt eigenlijk gecontroleerd door e...,Categorie AM: Bromfietsen met een maximumsnelh...,...,10,10,10,0,"1=Wasbeer,2=Wolf,3=Bruine beer,4=Das,5=Vos,6=B...",Management 1,Data science 1,<= 10 uren,2,Ja
4,10/02/2022 11:41:10,6/18,Rechts,43,173,B,positief,3,De wereld wordt eigenlijk gecontroleerd door e...,Categorie B: Wagens.,...,10,7,7,2,"1=Wolf,2=Vos,3=Bruine beer,4=Bizon,5=Panter,6=...",Management 1,Data science 1,<= 10 uren,nee :),Ja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,17/02/2022 12:19:54,8/18,Rechts,45,188,A,positief,1,,Categorie B: Wagens.,...,6,7,6,4,"1=Otter,2=Wasbeer,3=Vos,4=Bruine beer,5=Bever,...",Netwerken 1,Data science 1,16-20 uren,helemaal niets,Ja
88,17/02/2022 18:24:53,7/18,Rechts,43,176,B,positief,2,,Categorie B: Wagens.,...,4,9,3,9,"1=Poema,2=Panter,3=Wolf,4=Bruine beer,5=Bizon,...",Data science 1,Programmeren 1 - Java,26-30 uren,Okey haha,Ja
89,20/02/2022 17:58:30,5/18,Rechts,43,181,Weet ik niet,Weet ik niet,1,,Categorie B: Wagens.,...,10,6,8,5,"1=Bruine beer,2=Wolf,3=Bizon,4=Wasbeer,5=Otter...",Programmeren 1 - Java,Programmeren 1 - Java,11-15 uren,,Ja
90,21/02/2022 22:46:10,10/18,Rechts,45,185,A,positief,1,,Categorie B: Wagens.,...,9,7,7,0,"1=Wolf,2=Bruine beer,3=Panter,4=Vos,5=Poema,6=...",Management 1,Programmeren 1 - Java,21-25 uren,helemaal niets,Ja


## We verwerken de gegevens verder tot meerdere dataframes

In [3]:
from scripts.enquete_scripts import *

# door de rechtstreekse verbinding met Google Sheets zijn de datumformaten blijkbaar anders dan na download als CSV
# ook worden multiple answers nu gesplitst met komma's en niet met punt-komma's bij download als CSV
enquete_df, vragen_df, fruit_df, mascotte_df, complot_df, rijbewijzen_df, vervoer_df = preprocess(enquete_df)


In [4]:
vragen_df

,vraag,datatype
vraag,,
Timestamp,Timestamp,datetime64[ns]
v1,Wat was je score op de GapMinder test? Je hoe...,float64
v2,Met welke hand schrijf je?,category
v3,Wat is je schoenmaat?,float64
v4,Wat is je lichaamslengte (in cm)?,float64
v5,Wat is je bloedgroep: AB0?,category
v6,Wat is de resusfactor van je bloedgroep?,category
v7,Hoeveel broers / zussen heb je?,float64
v8,Vink alle beweringen aan die volgens aannemel...,object


### Nearest Neighbors

The intuition behind the KNN algorithm is one of the simplest of all the supervised machine learning algorithms. It simply calculates the distance of a new data point to **all other training data points**. The distance can be of any type e.g Euclidean or Manhattan etc. It then selects the K-nearest data points, where K can be any integer. Finally it assigns the data point to the class to which the majority of the K data points belong.

#### Pro's
1. It is **lazy learning algorithm** and therefore requires no training prior to making real time predictions. This makes the KNN algorithm much **faster** than other algorithms that require training e.g SVM, linear regression, etc.
2. New data points can be added to the training data set without having to re-train the algorithm.
3. It is **easy to implement** and therefore can be used in any type of machine learning project.
4. It requires minimally only two parameters to be set.

#### Cons
1. The KNN algorithm is **not scalable**. This means that the KNN algorithm is not suitable for large datasets, because it compares a new data point to **all** other training points.
2. Because of the **Curse of Dimensionality** it is also **not suitable for high dimensional data**.
3. It doesn't work well on categorical data since it is based on a some kind of distance metric.
4. Don't forget to **normalize** the data.


We create a dataset consisting of X and Y values without NA's.

In [72]:
Xy = pd.concat([enquete_df.filter(regex='v23'), enquete_df.v10], axis=1)
# Xy = pd.concat([enquete_df.filter(regex='v23'), enquete_df.v26], axis=1)
Xy = Xy.dropna()
X = Xy.iloc[:, :-1]
y = Xy.iloc[:, -1]

### Model Scores
How well does the model perform on the test data using cross-validation?

In [121]:
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

for k in range(1, 30):
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model, X, y, cv=6, scoring='f1_macro')
    print(f'{k} -> f-score: {scores.mean():.2f} (+/- {scores.std() * 2:.2f})')

# k = 9 gives the best mean accuracy and standard deviation
model = KNeighborsClassifier(n_neighbors=5)
cross_val_score(model, X, y, cv=6).mean()

1 -> f-score: 0.51 (+/- 0.13)
2 -> f-score: 0.50 (+/- 0.23)
3 -> f-score: 0.47 (+/- 0.20)
4 -> f-score: 0.46 (+/- 0.11)
5 -> f-score: 0.52 (+/- 0.20)
6 -> f-score: 0.53 (+/- 0.25)
7 -> f-score: 0.49 (+/- 0.16)
8 -> f-score: 0.47 (+/- 0.25)
9 -> f-score: 0.47 (+/- 0.19)
10 -> f-score: 0.44 (+/- 0.23)
11 -> f-score: 0.45 (+/- 0.24)
12 -> f-score: 0.43 (+/- 0.18)
13 -> f-score: 0.43 (+/- 0.23)
14 -> f-score: 0.39 (+/- 0.14)
15 -> f-score: 0.45 (+/- 0.31)
16 -> f-score: 0.42 (+/- 0.24)
17 -> f-score: 0.45 (+/- 0.22)
18 -> f-score: 0.39 (+/- 0.18)
19 -> f-score: 0.46 (+/- 0.20)
20 -> f-score: 0.39 (+/- 0.25)
21 -> f-score: 0.43 (+/- 0.20)
22 -> f-score: 0.37 (+/- 0.15)
23 -> f-score: 0.43 (+/- 0.24)
24 -> f-score: 0.37 (+/- 0.15)
25 -> f-score: 0.41 (+/- 0.24)
26 -> f-score: 0.40 (+/- 0.23)
27 -> f-score: 0.39 (+/- 0.24)
28 -> f-score: 0.38 (+/- 0.14)
29 -> f-score: 0.41 (+/- 0.24)


0.5499999999999999

In [132]:

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(train_X, train_y)

pd.DataFrame(confusion_matrix(test_y, model.predict(test_X), labels=y.unique()),
             columns=y.unique(), index=y.unique())

# of met pivot
pd.DataFrame(zip(model.predict(test_X), test_y), columns=['prediction', 'actual']).pivot_table(index='prediction',
                                                                                               columns='actual',
                                                                                               aggfunc=len)


actual,"buiten de stad (een dorp, een gehucht, het platteland, ...)",in een stad
prediction,,
"buiten de stad (een dorp, een gehucht, het platteland, ...)",7,1
in een stad,3,8


In [9]:
distances, indices = model.kneighbors(X, n_neighbors=5)
enquete_df.loc[indices[0, 1:]]

id,Timestamp,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v23c,v23d,v23e,v23f,v24,v25,v26,v27,v28,v29
47,2022-02-14 16:21:37,33.333333,Rechts,44.0,183.0,Weet ik niet,Weet ik niet,1.0,John F. Kennedy werd vermoord in opdracht van ...,Categorie B: Wagens.,...,8,5,5,6.0,"1=Panter,2=Poema,3=Vos,4=Wolf,5=Bruine beer,6=...",Management 1,User interfaces 1,18.0,,Ja
63,2022-02-17 10:53:53,16.666667,Rechts,42.5,180.0,Weet ik niet,Weet ik niet,5.0,De aanslagen van 9/11 in NY werden niet door t...,Categorie G: Landbouwvoertuigen.,...,9,7,6,3.0,"1=Otter,2=Panter,3=Poema,4=Wolf,5=Vos,6=Bruine...",Programmeren 1 - Java,Data science 1,10.0,,Ja
25,2022-02-10 21:11:18,61.111111,Rechts,41.0,168.0,Weet ik niet,Weet ik niet,1.0,Prinses Diana kwam niet per ongeluk om het leven,Categorie AM: Bromfietsen met een maximumsnelh...,...,7,8,5,4.0,"1=Vos,2=Bever,3=Wasbeer,4=Otter,5=Wolf,6=Bruin...",Geen enkel,Data science 1,18.0,,Ja
39,2022-02-14 14:40:31,44.444444,Links,42.0,180.0,O,negatief,2.0,De tsunami van 2004 in de Indische Oceaan was ...,Categorie B: Wagens.,...,6,6,7,5.0,"1=Panter,2=Poema,3=Wolf,4=Bizon,5=Bruine beer,...",Management 1,Programmeren 1 - Java,18.0,,Ja


### K-Nearest Neighbors Regression

Same principle as KNN but instead of assigning a class to a data point, it assigns a value to the data point.


In [159]:
from sklearn.metrics import SCORERS, r2_score, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

# predict person's height based on their shoe size
Xy = pd.concat([enquete_df.v3, enquete_df.v4], axis=1)

Xy = Xy.dropna()
X = Xy.iloc[:, :-1]
y = Xy.iloc[:, -1]

sorted(SCORERS.keys())

for k in range(1, 30):
    model = KNeighborsRegressor(n_neighbors=k)
    scores = cross_val_score(model, X, y, cv=6, scoring='neg_mean_squared_error')
    print(f'{k} -> r2: {scores.mean():.2f} (+/- {scores.std() * 2:.2f})')

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
model = KNeighborsRegressor(n_neighbors=16)
model.fit(train_X, train_y)

mean_squared_error(test_y, model.predict(test_X))


1 -> r2: -68.45 (+/- 42.41)
2 -> r2: -48.43 (+/- 46.49)
3 -> r2: -40.13 (+/- 29.46)
4 -> r2: -35.38 (+/- 27.96)
5 -> r2: -35.11 (+/- 25.40)
6 -> r2: -33.41 (+/- 24.06)
7 -> r2: -32.91 (+/- 25.73)
8 -> r2: -32.93 (+/- 21.80)
9 -> r2: -31.81 (+/- 24.24)
10 -> r2: -31.32 (+/- 23.93)
11 -> r2: -31.45 (+/- 24.31)
12 -> r2: -31.81 (+/- 24.05)
13 -> r2: -31.67 (+/- 25.95)
14 -> r2: -31.72 (+/- 27.66)
15 -> r2: -30.93 (+/- 26.38)
16 -> r2: -30.89 (+/- 26.34)
17 -> r2: -31.51 (+/- 26.46)
18 -> r2: -31.35 (+/- 27.12)
19 -> r2: -30.95 (+/- 28.08)
20 -> r2: -30.87 (+/- 26.65)
21 -> r2: -31.29 (+/- 26.71)
22 -> r2: -31.46 (+/- 27.19)
23 -> r2: -31.67 (+/- 26.84)
24 -> r2: -31.98 (+/- 26.73)
25 -> r2: -32.71 (+/- 26.51)
26 -> r2: -32.87 (+/- 27.14)
27 -> r2: -33.46 (+/- 27.30)
28 -> r2: -34.04 (+/- 27.72)
29 -> r2: -34.29 (+/- 28.47)


41.45497532894737